In [11]:
import pandas as pd
import numpy as np
import math
import datetime
import subprocess
import bandit.main as bd

In [12]:
def get_frame(filename):
    df = pd.read_excel(io=filename)
    # df_combined = pd.merge(df,df_lookup,on='Difficulty')
    # df_combined['Duration'] = df_combined['Time'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
    # df_combined['MatRate'] = df_combined['Mats'] / df_combined['Duration']
    # df_combined['DiffClass'] = df_combined['Class'] + '-' + df_combined['Difficulty']
    return df

In [13]:
def save_excel(df,col, madfunc):
    dfgb = df.groupby([col])
    dfgb = dfgb['Quality'].agg(['count','median', madfunc]).rename(columns={'mad_function': 'MAD'}).reset_index()
    print(dfgb)
    return dfgb

In [14]:
subprocess.Popen('EQTime.xlsx', shell=True)

In [15]:
df = get_frame('EQTime.xlsx')
df.tail(10)

,ProjeDate,Project,Duration
24,2018-08-10,PCAutoMechanic,00:27:42
25,2018-08-11,ShadowOfWar,00:49:46
26,2018-08-14,ShadowOfWar,00:26:27
27,2018-08-19,Beastlord,00:25:47
28,2018-08-24,PCAutoMechanic,00:52:19
29,2018-08-26,PCAutoMechanic,00:32:11
30,2018-08-31,ShadowOfWar,00:52:53
31,2018-09-03,ShadowOfWar,00:19:29
32,2018-09-08,DualBoxing,00:03:00
33,2018-09-16,ShadowOfWar,00:37:20


In [16]:
# reduce it
df = bd.reduce(df,'Project')
df['Duration'] = df['Duration'].apply(lambda x: pd.Timedelta(str(x))).astype('timedelta64[s]')
df_quality = bd.get_quality(df['Duration'])
df = df.join(df_quality)

23 0.9425 22


In [17]:
df.tail(10)

,ProjeDate,Project,Duration,Quality
24,2018-08-10,PCAutoMechanic,1662.0,0.481795
25,2018-08-11,ShadowOfWar,2986.0,0.912224
26,2018-08-14,ShadowOfWar,1587.0,0.457412
27,2018-08-19,Beastlord,1547.0,0.444408
28,2018-08-24,PCAutoMechanic,3139.0,0.961964
29,2018-08-26,PCAutoMechanic,1931.0,0.569246
30,2018-08-31,ShadowOfWar,3173.0,0.973017
31,2018-09-03,ShadowOfWar,1169.0,0.321521
32,2018-09-08,DualBoxing,180.0,0.000000
33,2018-09-16,ShadowOfWar,2240.0,0.669701


In [18]:
df.sort_values('Quality', ascending=False).head(10)

,ProjeDate,Project,Duration,Quality
21,2018-07-29,ShadowOfWar,3256.0,1.000000
30,2018-08-31,ShadowOfWar,3173.0,0.973017
28,2018-08-24,PCAutoMechanic,3139.0,0.961964
25,2018-08-11,ShadowOfWar,2986.0,0.912224
2,2018-04-29,Druid,2847.0,0.867035
23,2018-08-05,PCAutoMechanic,2559.0,0.773407
4,2018-04-29,Mage,2313.0,0.693433
6,2018-05-06,Witcher,2290.0,0.685956
33,2018-09-16,ShadowOfWar,2240.0,0.669701
1,2018-04-29,DualBoxing,2189.0,0.653121


In [19]:
dfgb = save_excel(df,'Project', bd.mad_function)

           Project  count    median       MAD
0             Bard      2  0.296651  0.139629
1        Beastlord      2  0.452536  0.008127
2            Druid      3  0.565670  0.058843
3       DualBoxing      3  0.408322  0.244798
4       ForTheKing      2  0.386378  0.018693
5         KerbalSP      2  0.325585  0.242360
6             Mage      3  0.413199  0.280234
7            Necro      2  0.253088  0.101918
8   PCAutoMechanic      4  0.671326  0.145806
9      ShadowOfWar      7  0.669701  0.242523
10         Witcher      3  0.378739  0.210663


In [20]:
df = df.merge(dfgb)
dfgb['total'] = dfgb['count'].sum()
dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'median', 'MAD'), axis=1)
dfgb.sort_values('donext', ascending=False)

,Project,count,median,MAD,total,donext
8,PCAutoMechanic,4,0.671326,0.145806,33,1.138800
1,Beastlord,2,0.452536,0.008127,33,1.113644
2,Druid,3,0.565670,0.058843,33,1.105462
4,ForTheKing,2,0.386378,0.018693,33,1.047486
9,ShadowOfWar,7,0.669701,0.242523,33,1.023078
5,KerbalSP,2,0.325585,0.242360,33,0.986693
0,Bard,2,0.296651,0.139629,33,0.957759
6,Mage,3,0.413199,0.280234,33,0.952991
3,DualBoxing,3,0.408322,0.244798,33,0.948115
10,Witcher,3,0.378739,0.210663,33,0.918531
